# Cuery usage example

In [11]:
import importlib
from pathlib import Path
from pprint import pprint

import instructor
import openai
import pandas as pd
from rich.pretty import pprint

from cuery import utils
from cuery.work import DirceJobs, DirceTasks

GDRIVE = Path("~/Google Drive/Shared drives/Solutions").expanduser()

# Jobs

In [2]:
DATA_DIR = GDRIVE / "Research/future_of_work/inputs/ine_dirce_aggregated_by_activity.csv"
data = pd.read_csv(DATA_DIR)
data.columns = [c.replace(" ", "_") for c in data.columns]
display(data)

n_examples = 5
context_df = data.iloc[:5]
context_dict = {col: context_df[col].tolist() for col in context_df.columns}
context_records = context_df.to_dict(orient="records")

,Division,Actividad_principal,Estimated_Employees_2024,Median_YoY_Growth_pct
0,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4
1,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2
2,43 Actividades de construcción especializada,"432 Instalaciones eléctricas, de fontanería y ...",303430,0.5
3,45 Venta y reparación de vehículos de motor y ...,451 Venta de vehículos de motor,102172,-0.5
4,46 Comercio al por mayor e intermediarios del ...,463 Comercio al por mayor de productos aliment...,325970,-0.9
5,46 Comercio al por mayor e intermediarios del ...,464 Comercio al por mayor de artículos de uso ...,228324,-2.0
6,"47 Comercio al por menor, excepto de vehículos...",471 Comercio al por menor en establecimientos ...,233708,1.3
7,"47 Comercio al por menor, excepto de vehículos...",477 Comercio al por menor de otros artículos e...,242341,-1.3
8,49 Transporte terrestre y por tubería,493 Otro transporte terrestre de pasajeros,142413,0.5
9,49 Transporte terrestre y por tubería,494 Transporte de mercancías por carretera y s...,273002,-1.5


In [3]:
jobs = await DirceJobs(context_df, model="gpt-4.1-mini", n_concurrent=2)
jobs

Gathering responses: 100%|██████████| 5/5 [00:11<00:00,  2.22s/it]


[Jobs(jobs=[Job(name='Quality Control Analyst', description='Responsible for reviewing and documenting quality metrics of meat products using software systems.', automation_potential=8, reason='Routine data analysis and pattern recognition with software.'), Job(name='Inventory Data Coordinator', description='Manages inventory records and updates stock databases related to meat processing supplies.', automation_potential=9, reason='High-volume data entry and database updates are easily automated.'), Job(name='Compliance Documentation Specialist', description='Prepares and maintains legal and safety compliance documents for meat processing operations.', automation_potential=7, reason='Standardized document preparation can be efficiently automated using AI.')]),
 Jobs(jobs=[Job(name='CAD Designer', description='Creates design plans for vehicle components using computer software.', automation_potential=7, reason='Repetitive design elements can be generated by AI with pattern recognition.')

In [4]:
jobs_df = DirceJobs.explode_responses(jobs, context_df)
jobs_df = jobs_df.rename(
    columns={
        "name": "job",
        "description": "job_description",
        "automation_potential": "job_automation_potential",
        "reason": "job_automation_reason",
    }
)
jobs_df

,Division,Actividad_principal,Estimated_Employees_2024,Median_YoY_Growth_pct,job,job_description,job_automation_potential,job_automation_reason
0,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Quality Control Analyst,Responsible for reviewing and documenting qual...,8,Routine data analysis and pattern recognition ...
1,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Inventory Data Coordinator,Manages inventory records and updates stock da...,9,High-volume data entry and database updates ar...
2,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Compliance Documentation Specialist,Prepares and maintains legal and safety compli...,7,Standardized document preparation can be effic...
3,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2,CAD Designer,Creates design plans for vehicle components us...,7,Repetitive design elements can be generated by...
4,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2,Quality Control Analyst,Analyzes data and documentation to ensure vehi...,6,Data evaluation tasks can be automated using A...
5,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2,Production Planner,Schedules manufacturing processes and coordina...,8,Scheduling and resource optimization can be ef...
6,43 Actividades de construcción especializada,"432 Instalaciones eléctricas, de fontanería y ...",303430,0.5,Project Estimator,Prepares cost estimates and budgets for constr...,7,Uses historical data and standard calculations...
7,43 Actividades de construcción especializada,"432 Instalaciones eléctricas, de fontanería y ...",303430,0.5,Procurement Specialist,Manages purchase orders and vendor communicati...,6,Routine vendor and order management tasks can ...
8,43 Actividades de construcción especializada,"432 Instalaciones eléctricas, de fontanería y ...",303430,0.5,Scheduling Coordinator,Creates and updates work schedules for constru...,7,Well-defined scheduling rules make this role a...
9,45 Venta y reparación de vehículos de motor y ...,451 Venta de vehículos de motor,102172,-0.5,Sales Clerk,Handles customer transactions and sales proces...,7,High potential for automation due to repetitiv...


In [ ]:
# jobs with Perplexity (using its OpenAI-compatible API)
import os

utils.set_api_keys()

perplex = openai.AsyncOpenAI(
    api_key=os.getenv("PERPLEXITY_API_KEY"), base_url="https://api.perplexity.ai"
)
perplex = instructor.from_perplexity(perplex)
jobs_perp = await DirceJobs(context_df, client=perplex, model="sonar", n_concurrent=2)

Gathering responses: 100%|██████████| 5/5 [06:37<00:00, 79.58s/it] 

Encountered: 5 response parsing errors!


In [19]:
DirceJobs.explode_responses(jobs_perp, context_df)

,Division,Actividad_principal,Estimated_Employees_2024,Median_YoY_Growth_pct,name,description,automation_potential,reason
0,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Inventory Management Clerk,"Tracks and manages stock levels, orders, and d...",9,"AI can automate stock tracking, ordering, and ..."
1,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Quality Assurance Data Analyst,Analyzes quality control data from production ...,9,"AI excels at processing large datasets, detect..."
2,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Order Processing & Customer Service Rep. (Digi...,"Handles order entry, customer inquiries via em...",8,AI-driven systems can process orders and respo...
3,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Regulatory Compliance Documentation Specialist,Prepares and maintains compliance documentatio...,7,AI tools can integrate regulatory updates and ...
4,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Production Planning and Scheduling Officer,Schedules production runs based on demand fore...,9,AI systems can analyze demand patterns and res...
5,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2,Design & CAD Technician,AI can automate vehicle component design tasks...,8,AI-driven design and simulation tools automate...
6,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2,Quality Control Inspector (Software-Based),AI detects defects in vehicle parts via softwa...,9,AI-based systems fully automate defect detecti...
7,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2,Supply Chain Data Analyst,AI forecasts demand and manages inventory auto...,8,AI software can handle predictive modeling and...
8,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2,Procurement Clerk,AI can automate purchase orders and vendor man...,7,"Routine, data-driven purchasing tasks are auto..."
9,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2,Production Planner,AI optimizes manufacturing schedules using rea...,8,AI excels at scheduling and planning using sof...


## Tasks

In [5]:
tasks = await DirceTasks(jobs_df, model="gpt-4.1-mini", n_concurrent=2)
tasks

Gathering responses: 100%|██████████| 15/15 [00:56<00:00,  3.77s/it]


[JobTasks(tasks=[JobTask(name='Data Entry and Report Generation', description='Automate inputting quality control data and generating regular quality reports.', automation_potential=8, intelligence=4, sexyness=5, scalability=8, data_needs='Requires quality control measurements and historical data for reports.', products=['Tableau', 'Power BI', 'Zoho Analytics']), JobTask(name='Defect Detection in Product Images', description='Use AI to analyze product images to identify defects or inconsistencies in meat products.', automation_potential=9, intelligence=7, sexyness=6, scalability=7, data_needs='Needs access to labeled product images with defects and quality benchmarks.', products=['Google Vision AI', 'Amazon Rekognition', 'IBM Watson Visual Recognition']), JobTask(name='Quality Parameter Monitoring', description='Automatically monitor and analyze quality parameters like temperature and pH during processing.', automation_potential=8, intelligence=6, sexyness=4, scalability=7, data_needs=

In [6]:
tasks_df = DirceTasks.explode_responses(tasks, jobs_df)
tasks_df = tasks_df.rename(columns={"name": "task", "description": "task_description"})
tasks_df

,Division,Actividad_principal,Estimated_Employees_2024,Median_YoY_Growth_pct,job,job_description,job_automation_potential,job_automation_reason,task,task_description,automation_potential,intelligence,sexyness,scalability,data_needs,products
0,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Quality Control Analyst,Responsible for reviewing and documenting qual...,8,Routine data analysis and pattern recognition ...,Data Entry and Report Generation,Automate inputting quality control data and ge...,8,4,5,8,Requires quality control measurements and hist...,"[Tableau, Power BI, Zoho Analytics]"
1,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Quality Control Analyst,Responsible for reviewing and documenting qual...,8,Routine data analysis and pattern recognition ...,Defect Detection in Product Images,Use AI to analyze product images to identify d...,9,7,6,7,Needs access to labeled product images with de...,"[Google Vision AI, Amazon Rekognition, IBM Wat..."
2,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Quality Control Analyst,Responsible for reviewing and documenting qual...,8,Routine data analysis and pattern recognition ...,Quality Parameter Monitoring,Automatically monitor and analyze quality para...,8,6,4,7,Requires real-time sensor data of processing e...,"[OSIsoft PI System, Honeywell Process Historia..."
3,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Quality Control Analyst,Responsible for reviewing and documenting qual...,8,Routine data analysis and pattern recognition ...,Compliance Documentation Automation,Generate compliance documentation and certific...,7,5,3,6,Quality control data and relevant regulatory s...,"[DocuSign, Adobe Sign, Formstack]"
4,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Inventory Data Coordinator,Manages inventory records and updates stock da...,9,High-volume data entry and database updates ar...,Automated Data Entry,Automatically input inventory data into comput...,9,4,5,9,Requires structured inventory data and input f...,"[UiPath, Automation Anywhere, ABBYY FlexiCapture]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,46 Comercio al por mayor e intermediarios del ...,463 Comercio al por mayor de productos aliment...,325970,-0.9,Customer Service Representative,Responds to customer inquiries and resolves is...,7,Routine queries and responses can be automated...,Order Processing,"Automate order entry, validation, and status u...",8,4,5,8,Sales orders and customer data for processing ...,"[Salesforce Einstein, SAP Conversational AI]"
68,46 Comercio al por mayor e intermediarios del ...,463 Comercio al por mayor de productos aliment...,325970,-0.9,Customer Service Representative,Responds to customer inquiries and resolves is...,7,Routine queries and responses can be automated...,Customer Inquiry Response,AI chatbots to handle common customer question...,9,6,7,9,Frequently asked questions and customer intera...,"[Zendesk Answer Bot, IBM Watson Assistant]"
69,46 Comercio al por mayor e intermediarios del ...,463 Comercio al por mayor de productos aliment...,325970,-0.9,Customer Service Representative,Responds to customer inquiries and resolves is...,7,Routine queries and responses can be automated...,Complaint Logging and Tracking,Automatically log and track customer complaint...,7,5,4,7,Complaint data and resolution status information.,"[Freshdesk, Zoho Desk]"
70,46 Comercio al por mayor e intermediarios del ...,463 Comercio al por mayor de productos aliment...,325970,-0.9,Customer Service Representative,Responds to customer inquiries and resolves is...,7,Routine queries and responses can be automated...,Inventory Status Updates,Automate communication of product availability...,8,4,6,8,Real-time inventory data and product catalogs.,"[TradeGeck

In [ ]:
tasks_df.to_csv(
    GDRIVE / "Research/future_of_work/outputs/ine_dirce_ai_tasks.csv",
    index=False,
)

In [9]:
pprint(DirceJobs.prompt)
pprint(DirceJobs.response.model_json_schema())

Prompt(
│   messages=[
│   │   Message(
│   │   │   role='system',
│   │   │   content="You're an analyst at the Spanish 'Instituo Nacional de Estadística' (INE) analyzing\ndata from its 'Directorio Central de Empresas' (DIRCE). Your objective is to analyze\ngroups of companies, identified by a sector ('Division') and a corresponding main activity\n('Actividad principal') in order to identify jobs within those companies that are likely to\nbe automatable by AI. Both 'Division' and 'Actividad_principal' are provided in Spanish and may\ninclude numeric IDs that you can ignore if you don't understand them. Always respond in English.\nOnly consider jobs that are computer- or paper-based and can be automated by AI using software\n(don't include jobs automatable by robots or other physical means)."
│   │   ),
│   │   Message(
│   │   │   role='user',
│   │   │   content='Please analyze the following jobs sector and identify jobs that are automatable by AI software.\nDivision: {{Division}}\nActividad principal: {{Actividad_principal}}'
│   │   )
│   ],
│   required=['Division', 'Actividad_principal']
)

{
│   '$defs': {
│   │   'Job': {
│   │   │   'properties': {
│   │   │   │   'name': {
│   │   │   │   │   'description': 'Name of the job role (job title, less than 50 characters)',
│   │   │   │   │   'maxLength': 50,
│   │   │   │   │   'minLength': 5,
│   │   │   │   │   'title': 'Name',
│   │   │   │   │   'type': 'string'
│   │   │   │   },
│   │   │   │   'description': {
│   │   │   │   │   'description': 'A short description of the job role (less than 200 characters)',
│   │   │   │   │   'maxLength': 200,
│   │   │   │   │   'minLength': 20,
│   │   │   │   │   'title': 'Description',
│   │   │   │   │   'type': 'string'
│   │   │   │   },
│   │   │   │   'automation_potential': {
│   │   │   │   │   'description': "A score from 1 to 10 indicating the job's potential for automation",
│   │   │   │   │   'maximum': 10,
│   │   │   │   │   'minimum': 0,
│   │   │   │   │   'title': 'Automation Potential',
│   │   │   │   │   'type': 'integer'
│   │   │   │   },
│   │   │   │   'reason': {
│   │   │   │   │   'description': 'A short explanation of no more than 10 words, of why the job is likely to be automatable with the given potential score.',
│   │   │   │   │   'maxLength': 300,
│   │   │   │   │   'minLength': 30,
│   │   │   │   │   'title': 'Reason',
│   │   │   │   │   'type': 'string'
│   │   │   │   }
│   │   │   },
│   │   │   'required': ['name', 'description', 'automation_potential', 'reason'],
│   │   │   'title': 'Job',
│   │   │   'type': 'object'
│   │   }
│   },
│   'properties': {
│   │   'jobs': {
│   │   │   'description': 'A list of jobs with their AI automation potential and reasons for that potential',
│   │   │   'items': {'$ref': '#/$defs/Job'},
│   │   │   'minItems': 3,
│   │   │   'title': 'Jobs',
│   │   │   'type': 'array'
│   │   }
│   },
│   'required': ['jobs'],
│   'title': 'Jobs',
│   'type': 'object'
}